# 2nd Feature Engineering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from matplotlib import rc, font_manager
rc("font", family="Malgun Gothic")

In [11]:
regular = pd.read_csv("data/Regular_Season_Batter.csv")

In [12]:
# lag 변수
for j in ['avg', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E']:
    lag_1 = []
    for i in range(len(regular)):
        if len(regular.loc[(regular["batter_name"]==regular["batter_name"].iloc[i])&(regular["year"]==regular["year"][i]-1)][j])==0:
            lag_1.append(np.nan)
        else:
            lag_1.append(regular.loc[(regular["batter_name"]==regular["batter_name"].iloc[i])&(regular["year"]==regular["year"][i]-1)][j].iloc[0])
    
    regular["lag_1_" + j] = lag_1
    print(j)

avg
G
AB
R
H
2B
3B
HR
TB
RBI
SB
CS
BB
HBP
SO
GDP
SLG
OBP
E


In [13]:
regular["1B"] = regular["H"] - regular["2B"] - regular["3B"] - regular["HR"]

In [14]:
regular.head()

,batter_id,batter_name,year,team,avg,G,AB,R,H,2B,...,lag_1_SB,lag_1_CS,lag_1_BB,lag_1_HBP,lag_1_SO,lag_1_GDP,lag_1_SLG,lag_1_OBP,lag_1_E,1B
0,0,가르시아,2018,LG,0.339,50,183,27,62,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45
1,1,강경학,2011,한화,0.000,2,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,강경학,2014,한화,0.221,41,86,11,19,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
3,1,강경학,2015,한화,0.257,120,311,50,80,7,...,0.0,0.0,13.0,2.0,28.0,1.0,0.349,0.337,6.0,67
4,1,강경학,2016,한화,0.158,46,101,16,16,3,...,4.0,3.0,40.0,5.0,58.0,3.0,0.325,0.348,15.0,10


신뢰성이 있는 지표 : 홈런, 삼진, 볼넷 (lag_변수들 이용)

신뢰성이 낮은 지표 : 3루타, avg, 1루타 (누적 성적 이용)

신뢰성이 낮은 지포 : 1B/AB, avg, 3B/AB => 이 변수들은 lag로 불충분 (lag + total)

In [42]:
regular.loc[(regular["batter_name"]=="강경학")&(regular["year"]<2016),"1B"].sum() / regular.loc[(regular["batter_name"]=="강경학")&(regular["year"]<2016),"AB"].sum()

0.20100502512562815

In [43]:
regular.loc[(regular["batter_name"]=="강경학")&(regular["year"]<2016),"3B"].sum() / regular.loc[(regular["batter_name"]=="강경학")&(regular["year"]<2016),"AB"].sum()

0.017587939698492462

In [44]:
a = regular.loc[(regular["batter_name"]=="강경학")&(regular["year"]<2016)]
(a["avg"] * a["AB"]).sum() / a["AB"].sum()

0.24857537688442213

In [50]:
total_1BnAB = []
total_3BnAB = []
total_avg = []
for i in range(len(regular)):
    a = regular.loc[(regular["batter_name"]==regular["batter_name"].iloc[i])&(regular["year"]<regular["year"].iloc[i])]
    if len(a) == 0:
        total_1BnAB.append(np.nan)
        total_3BnAB.append(np.nan)
        total_avg.append(np.nan)        
    else:
        total_1BnAB.append(a["1B"].sum() / a["AB"].sum())
        total_3BnAB.append(a["3B"].sum() / a["AB"].sum())
        total_avg.append((a["avg"] * a["AB"]).sum() / a["AB"].sum())

In [51]:
regular["total_1BnAB"] = total_1BnAB
regular["total_3BnAB"] = total_3BnAB
regular["total_avg"] = total_avg

## Model building

In [18]:
regular = pd.read_csv("data/Regular_Season_Batter.csv")
del regular["team"]
del regular["career"]

In [19]:
regular.columns

Index(['batter_id', 'batter_name', 'year', 'avg', 'G', 'AB', 'R', 'H', '2B',
       '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP', 'SLG',
       'OBP', 'E', 'height/weight', 'year_born', 'position', 'starting_salary',
       'OPS'],
      dtype='object')

In [20]:
agg = {}
for i in regular.columns:
    agg[i] = []
for i in regular.loc[regular["year"]==2018, "batter_name"].unique():
    for j in regular.columns:
        if j in ['batter_id','batter_name','height/weight','year_born','position','starting_salary']:
            agg[j].append(regular.loc[regular["batter_name"]==i,j].iloc[0])
        elif j in ['avg', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP', 'SLG', 'OBP', 'E', 'OPS']:
            agg[j].append(0)
        elif j == "year":
            agg[j].append(2019)

In [25]:
regular = pd.concat([regular, pd.DataFrame(agg, columns=regular.columns)])

In [27]:
regular.year.value_counts()

2019    227
2018    227
2017    217
2016    213
2015    207
2013    194
2014    186
2012    174
2011    151
2010    130
2009    124
2008    115
2007     98
2006     85
2005     73
2004     68
2003     54
2002     43
2001     32
2000     20
1999     14
1998     10
1997      8
1996      7
1994      2
1995      1
1993      1
Name: year, dtype: int64

In [28]:
regular["1B"] = regular["H"] - regular["2B"] - regular["3B"] - regular["HR"]

In [31]:
# lag 변수
for j in ['avg', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E']:
    lag_1 = []
    for i in range(len(regular)):
        if len(regular.loc[(regular["batter_name"]==regular["batter_name"].iloc[i])&(regular["year"]==regular["year"].iloc[i]-1)][j])==0:
            lag_1.append(np.nan)
        else:
            lag_1.append(regular.loc[(regular["batter_name"]==regular["batter_name"].iloc[i])&(regular["year"]==regular["year"].iloc[i]-1)][j].iloc[0])
    
    regular["lag_1_" + j] = lag_1
    print(j)

avg
G
AB
R
H
2B
3B
HR
TB
RBI
SB
CS
BB
HBP
SO
GDP
SLG
OBP
E


In [41]:
i = 1
regular["batter_name"].iloc[i], regular["year"].iloc[i]
regular.iloc[i,:]

batter_id                      1
batter_name                  강경학
year                        2011
avg                            0
G                              2
AB                             1
R                              0
H                              0
2B                             0
3B                             0
HR                             0
TB                             0
RBI                            0
SB                             0
CS                             0
BB                             0
HBP                            0
SO                             1
GDP                            0
SLG                            0
OBP                            0
E                              1
height/weight         180cm/72kg
year_born          1992년 08월 11일
position               내야수(우투좌타)
starting_salary          10000만원
OPS                            0
1B                             0
lag_1_avg                    NaN
lag_1_G                      NaN
lag_1_AB  

In [52]:
total_1BnAB = []
total_3BnAB = []
total_avg = []
for i in range(len(regular)):
    a = regular.loc[(regular["batter_name"]==regular["batter_name"].iloc[i])&(regular["year"]<regular["year"].iloc[i])]
    if len(a) == 0:
        total_1BnAB.append(np.nan)
        total_3BnAB.append(np.nan)
        total_avg.append(np.nan)        
    else:
        total_1BnAB.append(a["1B"].sum() / a["AB"].sum())
        total_3BnAB.append(a["3B"].sum() / a["AB"].sum())
        total_avg.append((a["avg"] * a["AB"]).sum() / a["AB"].sum())

In [53]:
regular["total_1BnAB"] = total_1BnAB
regular["total_3BnAB"] = total_3BnAB
regular["total_avg"] = total_avg

In [54]:
input_var = [x for x in regular.columns if ("lag_" in x)|("total_" in x)]

In [55]:
train = regular.loc[regular["year"]<=2018]
train = train.loc[-train["OPS"].isnull()]

test = regular.loc[regular["year"]==2019]

In [56]:
X = train[input_var]
y = train["OPS"]

In [59]:
import lightgbm as lgb

param = {"num_leaves": 100,
         "min_data_in_leaf": 30,
         "objective": "regression",
         "max_depth": 6,
         "learning_rate": 0.02,
         "boosting": "gbdt",
         "feature_fraction": 0.7522,
         "bagging_freq": 1,
         "bagging_fraction": 0.7083,
         "bagging_seed": 11,
         "metric": "rmse",
         "random_state": 133}
trn_data = lgb.Dataset(train[input_var], label=train["OPS"], weight=train["AB"])
num_round = 800
clf = lgb.train(param, trn_data, num_round, valid_sets=[trn_data], verbose_eval=0, early_stopping_rounds = 200)

In [60]:
predictions = clf.predict(test[input_var], num_iteration=clf.best_iteration)

In [61]:
predictions[:5]

array([ 0.72325523,  0.69846143,  0.63083104,  0.55778291,  0.81041556])

In [62]:
test[input_var].head()

,lag_1_avg,lag_1_G,lag_1_AB,lag_1_R,lag_1_H,lag_1_2B,lag_1_3B,lag_1_HR,lag_1_TB,lag_1_RBI,...,lag_1_BB,lag_1_HBP,lag_1_SO,lag_1_GDP,lag_1_SLG,lag_1_OBP,lag_1_E,total_1BnAB,total_3BnAB,total_avg
0,0.339,50.0,183.0,27.0,62.0,9.0,0.0,8.0,95.0,34.0,...,9.0,8.0,25.0,3.0,0.519,0.383,9.0,0.245902,0.000000,0.339000
1,0.278,77.0,245.0,42.0,68.0,11.0,1.0,5.0,96.0,27.0,...,38.0,4.0,59.0,7.0,0.392,0.382,2.0,0.188406,0.013285,0.242726
2,0.200,4.0,5.0,0.0,1.0,1.0,0.0,0.0,2.0,1.0,...,0.0,0.0,3.0,0.0,0.400,0.200,0.0,0.000000,0.000000,0.125000
3,0.250,6.0,4.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.250,0.250,0.0,0.062500,0.000000,0.124688
4,0.269,129.0,427.0,46.0,115.0,14.0,0.0,22.0,195.0,71.0,...,29.0,14.0,96.0,11.0,0.457,0.331,8.0,0.179735,0.001894,0.276548


In [69]:
sub = test[["batter_id", "batter_name"]]
sub.head()

,batter_id,batter_name
0,0,가르시아
1,1,강경학
2,2,강구성
3,3,강민국
4,4,강민호


In [70]:
sub["ops"] = predictions

In [71]:
sub.to_csv("result/submission.csv", index=False)

In [74]:
sub = pd.read_csv("result/submission.csv", engine="python")
sub.head()

,batter_id,batter_name,ops
0,0,가르시아,0.723255
1,1,강경학,0.698461
2,2,강구성,0.630831
3,3,강민국,0.557783
4,4,강민호,0.810416


In [84]:
sub.sort_values(by=["ops"], ascending=False)[0:10]

,batter_id,batter_name,ops
40,58,김재환,1.049841
80,113,박병호,1.024168
68,91,로맥,1.011138
67,90,러프,1.007314
207,313,최정,0.988078
221,334,호잉,0.980193
69,95,로하스,0.972761
216,326,한동민,0.970000
106,149,손아섭,0.964248
140,202,이대호,0.964025
